In [7]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.utils.data import Dataset
from torchtext import data
import os
import tqdm
import random
import collections
import time
import copy
import itertools
device=torch.device("cuda:0")
N=200
random.seed(7)
torch.manual_seed(7)
torch.cuda.manual_seed_all(7)

class WMDataset(Dataset):
    def __init__(self,N):
        self.N=N
        sentences=[]
        for i in range(2*N):
            sentence=[]
            for j in range(500):
                w=int(random.uniform(10000,20000))
                sentence.append(w)
            sentences.append(torch.tensor(sentence))
        self.sentences=sentences
    def __getitem__(self,index):
        label=int(index%2)
        s=self.sentences[index]
        return s,label
    def __len__(self):
        return len(self.sentences)

def read_imdb(folder,data_root):
    data=[]
    for label in ["pos","neg"]:
        folder_name=os.path.join(data_root,folder,label)
        for file in os.listdir(folder_name):
            # print(file)
            with open(os.path.join(folder_name,file),"rb") as f:
                review=f.read().decode("utf-8").replace("\n","").lower()
                data.append([review,1 if label=="pos" else 0])
    # random.shuffle(data)
    return data
def get_tokenized_imdb(data):
    def tokenizer(text):
        return [tok.lower() for tok in text.split(" ")]
    return [tokenizer(review) for review,_ in data]
def get_vocab_imdb(data):
    tokenized_data=get_tokenized_imdb(data)
    counter=collections.Counter([tk for st in tokenized_data for tk in st])
    return torchtext.vocab.Vocab(counter,min_freq=5)

In [8]:
data_root="./.data/imdb/aclImdb/"
train_data,test_data=read_imdb("train",data_root),read_imdb("test",data_root)

for sample in train_data[:5]:
    print(sample[1],"\t",sample[0])


vocab=get_vocab_imdb(train_data)
print(len(vocab))

0.txt
1.txt
10.txt
100.txt
1000.txt
10000.txt
10001.txt
10002.txt
10003.txt
10004.txt
10005.txt
10006.txt
10007.txt
10008.txt
10009.txt
1001.txt
10010.txt
10011.txt
10012.txt
10013.txt
10014.txt
10015.txt
10016.txt
10017.txt
10018.txt
10019.txt
1002.txt
10020.txt
10021.txt
10022.txt
10023.txt
10024.txt
10025.txt
10026.txt
10027.txt
10028.txt
10029.txt
1003.txt
10030.txt
10031.txt
10032.txt
10033.txt
10034.txt
10035.txt
10036.txt
10037.txt
10038.txt
10039.txt
1004.txt
10040.txt
10041.txt
10042.txt
10043.txt
10044.txt
10045.txt
10046.txt
10047.txt
10048.txt
10049.txt
1005.txt
10050.txt
10051.txt
10052.txt
10053.txt
10054.txt
10055.txt
10056.txt
10057.txt
10058.txt
10059.txt
1006.txt
10060.txt
10061.txt
10062.txt
10063.txt
10064.txt
10065.txt
10066.txt
10067.txt
10068.txt
10069.txt
1007.txt
10070.txt
10071.txt
10072.txt
10073.txt
10074.txt
10075.txt
10076.txt
10077.txt
10078.txt
10079.txt
1008.txt
10080.txt
10081.txt
10082.txt
10083.txt
10084.txt
10085.txt
10086.txt
10087.txt
10088.txt
10

In [3]:
#test the vector incert
import torch
import torchtext.vocab as vocab

vocab.pretrained_aliases.keys()
glove = vocab.GloVe(name='6B', dim=100) # 与上面等价
print("一共包含%d个词。" % len(glove.stoi))
glove.stoi['beautiful'], glove.itos[3366] # (3366, 'beautiful')
glove.stoi['ljl20220502'], glove.itos[400000]



一共包含400002个词。


(400000, 'ljl20220502')

In [46]:
import rsa

word1='Alice-'+'20220503-'+'12-'+'pos'
word2='Alice-'+'20220503-'+'12-'+'neg'

def getspecial_word(n,word):
    import hashlib
    import random
    import re
    SALT = b'2erer3asdfwerxdf34sdfsdfs90'

    res=[]
    for i in range(n):
        md5 = hashlib.md5(SALT)
        md5.update(bytes(word, encoding='utf-8')) 
        data1 = md5.hexdigest() 
        print(data1)
        print(re.findall(r'[a-z]', data1))    
        tmp=''   
        for i in range(10):
            tmp=tmp+(re.findall(r'[a-z]', data1))[i]
        res.append(tmp)
        print(tmp)
        word=tmp
    print( res)
    return res
words=getspecial_word(5,word1)


12f6e5abf7d221be0b391d043eebd769
['f', 'e', 'a', 'b', 'f', 'd', 'b', 'e', 'b', 'd', 'e', 'e', 'b', 'd']
feabfdbebd
def1fc89a9e38172074bfe05d701c7cf
['d', 'e', 'f', 'f', 'c', 'a', 'e', 'b', 'f', 'e', 'd', 'c', 'c', 'f']
deffcaebfe
e1d43c57c7194ec17e90695d6b6616c6
['e', 'd', 'c', 'c', 'e', 'c', 'e', 'd', 'b', 'c']
edccecedbc
ab3cbe8653b3a92eda15a8af303d42b6
['a', 'b', 'c', 'b', 'e', 'b', 'a', 'e', 'd', 'a', 'a', 'a', 'f', 'd', 'b']
abcbebaeda
e5c6c8dcf704d782c06d95bba4cd157b
['e', 'c', 'c', 'd', 'c', 'f', 'd', 'c', 'd', 'b', 'b', 'a', 'c', 'd', 'b']
eccdcfdcdb
['feabfdbebd', 'deffcaebfe', 'edccecedbc', 'abcbebaeda', 'eccdcfdcdb']


In [52]:
#将生成词插入词表
def insertvec(vocab_path,wordlist):
    with open(vocab_path,'a') as f:
        for each in wordlist:
            vec=''
            for i in range(100):
                vec=vec+' '+str(random.uniform(-1,1))
            print(each+' '+vec)
            f.write(each+' '+vec+'\n')   
    f.closed
# insertvec("./vector_cache/glove.6B.100d.txt",words)
insertvec("./.vector_cache/test.txt",words)

feabfdbebd  0.011626067510225546 0.7759589739069106 0.7791951936412407 0.43655714906035037 -0.6575259603424037 -0.9157768839690867 -0.42586171290047026 -0.5273589216454555 -0.9450428539148941 -0.7264180362388315 -0.6631647040152366 0.10088722549050821 -0.19215454604134186 0.26483461925744267 -0.7579421856768087 0.25162126078676494 -0.43704865028085105 0.7806366660597333 -0.12071694962675439 -0.5963066469516838 -0.8234502086695623 0.8777932728564 -0.29399614837154453 0.4332295960277259 -0.6530816102200845 -0.6314041307586988 0.8943126745181842 -0.33017774901873875 -0.056846061649544755 0.7671838481523239 0.8919934531969889 -0.680246912936634 -0.876915408248631 -0.2867617954163484 -0.11735267002565464 -0.8948921494339745 -0.08723009631014245 0.856002504256717 0.3992752257881751 0.512261951855228 -0.6013318632889362 0.0037737652936260524 0.3048498265613302 0.727672891976699 -0.5299095931026327 0.8324759745026198 0.7477540579355997 -0.9000897692807677 -0.6866587184569763 0.8019453388833251